<a href="https://colab.research.google.com/github/SathishKasi/Physics-Enabled-Machine-Learning-Grinding/blob/main/Input_Parameter_Iteration_CESMII_Grinding_V5_Google_Collab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necessary dependencies including numpy and tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### In addition to Python & Jupyter notebooks, certain dependencies have to be installed for this model. That is what we shall do here.

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install tensorflow

ERROR: Operation cancelled by user


## Importing all necessary Python libraries

In [ ]:
import numpy as np
from keras.models import load_model

## Total stock value and Q'w range

### The roughing stock on the part in inches and the Q'w limits are the primary inputs to the algorithm and hence we require them first.

In [ ]:
roughing_stock = 0.0285
#Minimum value of Q'w
min_qw = 5
#Maximum value of Q'w.
#This value can only be set after the successful test which does not cause harm to the part or to the quality of the part.
max_qw = 6

### The max Q'w value can only be set after the successful test which does not cause harm to the part or to the quality of the part.


## The variation in dimension for the roughing operation is determined as an intermediate input

In [ ]:
# Based on physics relationships CNC Kapps Niles Cylindrical Grinding. This equation should be fixed.
var_dim = roughing_stock*5.26

### The constant 5.26 is an empirical constant for the KAPP-NILES ZP 40 and ZP i25 machines alone.

## The number of cycles and revolutions limits can be altered here if required.

In [ ]:
# The values of revolutions and cycles must be integers
revolution = [1,2,3]
cycle = [1,2,3,4]

## The program creates an empty list to store all eligible radial infeeds, revolutions, and cycles' values

In [ ]:
radial_infeed_list = []
revolution_list =[]
cycle_list = []

## Determination of eligible radial infeed, revolution, and cycle values is done inline with modeled constraints

In [ ]:
for i in range(len(revolution)):
    for j in range(len(cycle)):
# Based on physics relationships of CNC Kapps Niles Cylindrical Grinding. These equations should be fixed.
        com = revolution[i]*cycle[j]
        LF_stock = var_dim / (10.52 * com)
        radial_infeed = LF_stock * 2.39
#The radial_infeed value should be smaller than 0.005 and number of cycles can't be smaller than number of revolution
        if radial_infeed < 0.005 and cycle[j] >= revolution[i]:
# Add all eligible radial infeed, revolution, and cycle values to the coressponding list for the further processing.
            radial_infeed_list.append(round(radial_infeed,4))
            revolution_list.append(revolution[i])
            cycle_list.append(cycle[j])

### The constants 10.52 and 2.39 are empirical constants for the KAPP-NILES ZP 40 and ZP i25 machines alone.

## Creating an empty list to store feed rate and further optimize radial infeed, revolution, and cycle values with Q'w constraints

In [ ]:
feed_rate_list = []
radial_infeed_list_update = []
revolution_list_update = []
cycle_list_update = []

In [ ]:
#k is the feed rate value, whose range is between 130 and 200
for k in range(130,200,1):
    for n in range(len(radial_infeed_list)):
# Q'w value range constraints. This equation should be fixed.
        if min_qw< k * radial_infeed_list[n] * 10.72 < max_qw:
# Add all eligible feed rate, radial infeed, revolution, and cycle values to the coressponding list. These lists are the final version, which is part of the hybrid model input.
            feed_rate_list.append(k)
            radial_infeed_list_update.append(radial_infeed_list[n])
            revolution_list_update.append(revolution_list[n])
            cycle_list_update.append(cycle_list[n])

### The constant 10.72 is an empirical constant for the KAPP-NILES ZP 40 and ZP i25 machines alone.

## Generate physics output list

In [ ]:
physics_output_list =[]
for m in range(len(radial_infeed_list_update)):
# Based on physics relationships of CNC Kapps Niles Cylindrical Grinding, we calculate the output of the physical model.
    physics_output = radial_infeed_list_update[m] * feed_rate_list[m]/60
    physics_output_list.append(physics_output)

## Generate total stock list

In [ ]:
# Create the list that contains all total stock value
Total_Stock_list = []

for o in range(len(physics_output_list)):
    Total_Stock_list.append(roughing_stock)

## Merge six input parameters together

In [ ]:
# Now we have 6 saperate list for all input parameters and the last step is to merge them together.
Model_Input_Data =np.stack((Total_Stock_list,radial_infeed_list_update,feed_rate_list,cycle_list_update,revolution_list_update,physics_output_list), axis=1)

## Loading pretrained hybrid model parameters and make predictions (first value is total energy and second value is total time)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# The location of the pre-trained model file. Please change it accordingly on your desktop.
model = load_model('/content/drive/My Drive/bottom_infusion.h5')
# The input contains two parts. The first part contains five parameters (total stock, radial infeed, feed rate, number of cycle, number of revolution). The second part is the output of the physical model (single value).
predictions = model.predict([Model_Input_Data[:,:5],Model_Input_Data[:,5:6]])
# In order to make hybrid model training convergence fast, the unit of energy is set as kilowatt hour. Now we transfer it to joule.
predictions[:,0] = predictions[:,0]*3600000


2/2 [==============================] - 0s 11ms/step


### The file with the extension 'h5' should be in the same folder as this Python notebook file. Please do not rename that file.

## Obtaining the minimum values of energy and time from the prediction values

In [ ]:
min_row = np.where(predictions==np.min(predictions[:,0]))[0]

## Printing the minimum energy/time values and corresponding 6 input parameters values  

In [ ]:
print("\n")
print("RESULTS")
print("\n")
print("The PREDICTED minimum time consumption is %d minutes " % predictions[min_row,1])
print("\n")
print("The PREDICTED minimum energy consumption is %d joules" % predictions[min_row,0] )
print("The PREDICTED minimum energy consumption is %d kWH" % (predictions[min_row,0]/3600000) )
print("\n")
print("The affectable process parameters that can lead to the lowest time and energy are:" )
print("\n")
print("The ROUGHING stock is %g inches" % Model_Input_Data[min_row,0])
print("The radial infeed is %g inches" % Model_Input_Data[min_row,1])
print("The Feed rate is %g inches/min" % Model_Input_Data[min_row,2])
print("The number of cycles is %g cycles" % Model_Input_Data[min_row,3])
print("The number of revolutions is %g revolutions" % Model_Input_Data[min_row,4])



RESULTS


The PREDICTED minimum time consumption is 188 minutes 


The PREDICTED minimum energy consumption is 498122080 joules
The PREDICTED minimum energy consumption is 138 kWH


The affectable process parameters that can lead to the lowest time and energy are:


The total stock is 0.0285 inches
The radial infeed is 0.0038 inches
The Feed rate is 147 inches/min
The number of cycles is 3 cycles
The number of revolutions is 3 revolutions


Please note that these are suggested process parameters for the highest Q'w value set in a cell above.

Please update the maximum value in case you want to view proposed settings for a new Q'w value

Any increase in a Q'w value MUST be tested on a SCRAP part in real time before using the same conditions on a production part